In [36]:
from ibis import *
from nltk import parse

######################################################################
# CFG grammar based on NLTK
######################################################################

class CFG_Grammar(Grammar):
    """CFG parser based on NLTK."""
    
    def loadGrammar(self, grammarFilename):
        self.parser = parse.load_parser(grammarFilename, trace=1, cache=False)

    def interpret(self, user_input):
        """Parse an user_input string into a dialogue move or a set of moves."""
        try: return self.parseString(user_input)
        except: pass
        try: return eval(user_input)
        except: pass
        return set([])

    def parseString(self, user_input):
        tokens = user_input.split()
        try:
            for tree in self.parser.parse(tokens):
                print()
                print("---> 1trees[{}]".format(tree))
                print()
#                 sem_value = ""
#                 sem_value = tree
#             print("---> sem_value[{}]".format(sem_value))
            trees = next(self.parser.parse(tokens))
            for val in trees:
                print()
                print("---> 2trees[{}]".format(val))
                print()
            sem = trees[0].label().get('sem')
            print("trees[0].label()=++{}++".format(sem))
        except Exception as ex: # 에러 종류
            print('parseString에서 에러가 발생 했습니다\n', ex)
        return self.sem2move(sem)

    def sem2move(self, sem):
        try: 
            return Answer(sem['Answer'])
        except: pass
        try:
            ans = sem['Answer']
            pred = ans['pred']
            ind = ans['ind']
            return Answer(pred+"("+ind+")")
        except: pass
        try: 
            return Ask(WhQ(Pred1(sem['Ask'])))
        except: pass
        print("sem2move(self, sem) --> None")
        return None

grammar = CFG_Grammar()
grammar.loadGrammar("file:travel_test.fcfg")
try:
    promptStr = input("U> ")
except EOFError:
    print("EOF")
    sys.exit()
move_or_moves = grammar.interpret(promptStr)
if not move_or_moves:
    print("Did not understand:", promptStr)
    print()
elif isinstance(move_or_moves, Move):
    print("type(move_or_moves)=[{}]".format(type(move_or_moves)))
    print("move_or_moves=[{}]".format(move_or_moves))
    print()
else:
    print("type(move_or_moves)=[{}]".format(type(move_or_moves)))
    print("move_or_moves=[{}]".format(move_or_moves))
    print()




U> price
|.  price  .|
|[=========]| [0:1] 'price'
|[=========]| [0:1] WHQ[q='price'] -> 'price' *
|[=========]| [0:1] ASK[sem=[Ask='price']] -> WHQ[q='price'] *
|[=========]| [0:1] USR[sem=[Ask='price']] -> ASK[sem=[Ask='price']] *

---> 1trees[(USR[sem=[Ask='price']]
  (ASK[sem=[Ask='price']] (WHQ[q='price'] price)))]

|.  price  .|
|[=========]| [0:1] 'price'
|[=========]| [0:1] WHQ[q='price'] -> 'price' *
|[=========]| [0:1] ASK[sem=[Ask='price']] -> WHQ[q='price'] *
|[=========]| [0:1] USR[sem=[Ask='price']] -> ASK[sem=[Ask='price']] *

---> 2trees[(ASK[sem=[Ask='price']] (WHQ[q='price'] price))]

trees[0].label()=++[ Ask = 'price' ]++
type(move_or_moves)=[<class 'ibis_types.Ask'>]
move_or_moves=[Ask('?x.price(x)')]

